In [ ]:
import time
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [ ]:
from unet import unet

In [ ]:
gpu_enabled = torch.cuda.is_available()

In [ ]:
means = [0.485, 0.456, 0.406]  # mean of the imagenet dataset for normalizing

In [ ]:
stds = [0.229, 0.224, 0.225] # standard deviance of the imagenet dataset for normalizing

In [ ]:
def deNormalize(img,means=means,stds=stds):
    red = img[...,0] * stds[0] + means[0]
    green = img[...,1] * stds[1] + means[1]
    blue = img[...,2] * stds[2] + means[2]
    denormed = np.stack([red,green,blue],axis=2)
    return denormed

Generate dataset

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Resize((224,224)), transforms.Normalize(means,stds) ])
transform_target = transforms.Compose([transforms.ToTensor(),transforms.Resize((224,224)) ])

In [ ]:
train_data = datasets.VOCSegmentation(root='./Data', image_set='train', download=True, transform=transform, target_transform=transform_target)
val_data = datasets.VOCSegmentation(root='./Data', image_set='val', download=True, transform=transform, target_transform=transform_target)

In [ ]:
print(train_data)
print(val_data)

In [ ]:
train_loader = DataLoader(train_data, batch_size=10, shuffle=True,pin_memory=gpu_enabled)
val_loader = DataLoader(val_data, batch_size=10, shuffle=False,pin_memory=gpu_enabled)

rain step

In [ ]:
def train_step(train_loader,model,criterion,opt,train_losses,train_corr):
    model.train()
    for b,(img,label) in enumerate(train_loader):
        if gpu_enabled:
            img = img.cuda()
            label = label.cuda()
        b += 1
        opt.zero_grad()
        y = model(img)
        loss = criterion(y,torch.squeeze(label,dim=1))
        loss.backward()
        opt.step()
        if b%10 == 0:
            print(f"Batch : {b} , Train Loss : {loss} Train Acc ")
    train_losses.append(loss.item())

In [ ]:
 
#simply used to visualize a few examples
for b,(img_batch,label_batch) in enumerate(train_loader):
    break

In [ ]:
img = deNormalize(np.transpose(img_batch[0,...],(1,2,0)))
label = np.squeeze(np.transpose(label_batch[0,...],(1,2,0)),axis=2)

In [ ]:
def plot_example(img,label):
    plt.figure()
    plt.subplot(1,2,1)
    plt.imshow(img)
    plt.subplot(1,2,2)
    plt.imshow(label)

In [ ]:
plot_example(img,label)

In [ ]:
if gpu_enabled:
    model = unet(3,20).cuda()
else:
    model = unet(3,20)

In [ ]:
train_losses = []
train_corr = []

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
opt = torch.optim.Adam(model.parameters(),lr = 0.001)

In [ ]:
EPOCHS = 2

In [ ]:
for epoch in range(EPOCHS):
    train_step(train_loader,model,criterion,opt,train_losses,train_corr)

imply used to visualize a few examples

In [ ]:
for img_batch,label_batch in val_loader:
    break

In [ ]:
img = deNormalize(np.transpose(img_batch[0,...],(1,2,0)))
label = np.squeeze(np.transpose(label_batch[0,...],(1,2,0)),axis=2)
plot_example(img,label)

In [ ]:
with torch.no_grad():
    y = model(img_batch[0,...].view(-1,...))

In [ ]:
y = deNormalize(np.transpose(y[0,...],(1,2,0)))
plot_example(y,label)

In [ ]:
print(train_losses)